In [1]:
import sys
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

from libra_py import LoadPT
from libra_py import LoadMolecule
from libra_py import nve_md
from libra_py import units

import py3Dmol   # molecular visualization

U = Universe()
LoadPT.Load_PT(U, "elements.dat", 0)

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

## 1. Regular optimization example


Lets load the system from the file and save it as the first snapshot

In [2]:
syst = System()    
LoadMolecule.Load_Molecule(U, syst, "test1a.pdb", "pdb_1")    

xyz = nve_md.syst2xyz(syst)
#print xyz

24


Lets run the optimization recipe on this molecule and save the final snapshot

In [3]:
nve_md.optimize_syst(syst, {})

xyz2 = nve_md.syst2xyz(syst)
#print xyz2

Using the initial and final snapshots, lets animate the transition between the two structures

In [4]:
xyzall = xyz+xyz2

view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)\n
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(xyzall, 'xyz')
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'backandforward', 'step':1})
view.show() 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 2. Capping example

Lets do all the same, but for the system with F atom removed

In [5]:
syst = System()    
LoadMolecule.Load_Molecule(U, syst, "test1b.pdb", "pdb_1")    
xyz = nve_md.syst2xyz(syst)

22


Now, lets try to attach a new atom and optimize the resulting structure

Beware: if one optimizes the original structure and then add new atoms and optimize again - this doesn't work correctly.
So, for now, we need to first guess the location of the capping atom first, then add it to the system and only then - optimize.

In [6]:
rnd = Random()
d = 2.0 * units.Angst

x, y, z = 0.205* units.Angst,  -1.826* units.Angst,   1.258* units.Angst - d
syst.CREATE_ATOM( Atom(U, {"Atom_element":"F", "Atom_cm_x":x, "Atom_cm_y":y, "Atom_cm_z":z }  )  )
syst.LINK_ATOMS(8, 11)

xyz0 = nve_md.syst2xyz(syst)
print( xyz0)

 11 

C  2.409 0.661 0.000
H  3.084 1.614 1.652
H  3.084 1.614 -1.652
H  0.387 0.661 0.000
C  3.379 -2.082 0.000
H  2.706 -3.035 -1.652
H  5.401 -2.082 -0.000
C  2.409 -3.454 2.375
H  3.086 -2.504 4.027
H  0.387 -3.451 2.377
F  0.387 -3.451 -1.402



Lets finally optimize the guessed structure.

The option "fixed_fragment_translation" is used to not move all the atoms other than the newly added one

In [7]:
nve_md.optimize_syst(syst, {"anneal_schedule":[ [1.0, 10, 100], [10.0, 10, 100] ], "fixed_fragment_translation":range(0,10) })
xyz3 = nve_md.syst2xyz(syst)
print( xyz3)

 11 

C  2.409 0.661 0.000
H  3.084 1.614 1.652
H  3.084 1.614 -1.652
H  0.387 0.661 0.000
C  3.379 -2.082 0.000
H  2.706 -3.035 -1.652
H  5.401 -2.082 0.000
C  2.409 -3.454 2.375
H  3.086 -2.504 4.027
H  0.387 -3.451 2.377
F  3.345 -6.178 2.385



Visualize the result

In [8]:
xyzall2 = xyz0+xyz3

view = py3Dmol.view(width=800,height=400, linked=False,viewergrid=(1,3)) 
view.setBackgroundColor('0xeeeeee')                                     
view.zoomTo()                                                           
view.addModelsAsFrames(xyzall2, 'xyz', viewer=(0,0))
view.addModel(xyz0,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}}, viewer=(0,1))
view.addModel(xyz3,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}}, viewer=(0,2))
view.setStyle({'sphere':{'colorscheme':'Jmol', }})                        
view.animate({'reps':0, 'loop':'backandforth', 'step':1})
view.show() 

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol